# Feature selection

From this guide: https://scikit-learn.org/stable/modules/feature_selection.html

## Overview

The classes in the `sklearn.feature_selection` module can be used for feature selection/dimensionality reduction on sample sets, either to improve estimators’ accuracy scores or to boost their performance on very high-dimensional datasets.

## [Removing features with low variance](https://scikit-learn.org/stable/modules/feature_selection.html#removing-features-with-low-variance)

`VarianceThreshold` is a simple baseline approach to feature selection. 

It removes *all features whose variance doesn’t meet some threshold*. 

By **default**, it removes **all zero-variance features**, i.e. features that have the same value in all samples.

As an example, suppose that we have a dataset with boolean features, and we want to remove all features that are either one or zero (on or off) in more than 80% of the samples. Boolean features are Bernoulli random variables, and the variance of such variables is given by:

$
\text{Var}[X] = p(1-p)
$

so we can select using the threshold `.8 * (1 - .8)`:

In [2]:
from IPython.display import display
import numpy as np
from sklearn.feature_selection import VarianceThreshold

X = np.array(
    [
        [0, 0, 1], 
        [0, 1, 0], 
        [1, 0, 0], 
        [0, 1, 1], 
        [0, 1, 0], 
        [0, 1, 1]
    ]
)
print("X:\n", X)

X:
 [[0 0 1]
 [0 1 0]
 [1 0 0]
 [0 1 1]
 [0 1 0]
 [0 1 1]]


In [5]:
threshold = .8 * (1 - .8)
print(f"threshold = .8 * (1 - .8) = {threshold:.2f}")

sel = VarianceThreshold(threshold=threshold)  # NOTE: threshold parameter is a *variance* threshold.
sel.fit_transform(X)

threshold = .8 * (1 - .8) = 0.16


array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])

As expected, `VarianceThreshold` has removed the first column, which has a probability $p = 5/6 > .8$ of containing a zero.

## [Univariate feature selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)

Univariate feature selection works by *selecting the best features based on **univariate statistical tests***. 

It can be seen as a preprocessing step to an estimator. Scikit-learn exposes feature selection routines as objects that implement the `transform` method:
* `SelectKBest` removes all but the $k$ highest scoring features
* `SelectPercentile` removes all but a user-specified highest scoring percentage of features
* using common univariate statistical tests for each feature: false positive rate `SelectFpr`, false discovery rate `SelectFdr`, or family wise error `SelectFwe`.
* `GenericUnivariateSelect` allows to perform univariate feature selection with a configurable strategy. This allows to select the best univariate selection strategy with hyper-parameter search estimator.

For instance, we can perform a $\chi^2$ test to the samples to retrieve only the two best features as follows:

In [6]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [7]:
X, y = load_iris(return_X_y=True)
X.shape

(150, 4)

In [8]:
X_new = SelectKBest(
    score_func=chi2,  # We are providing a score function here!
    k=2
).fit_transform(X, y)
X_new.shape

(150, 2)

These objects take as input a scoring function that returns *univariate scores* and *p-values* (or only scores for `SelectKBest` and `SelectPercentile`):
* For regression: [`f_regression`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression), [`mutual_info_regression`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression)
* For classification: [`chi2`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2), [`f_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif), [`mutual_info_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)
> 📚🎓 See the documentation for each of the above functions for more details.

The methods based on F-test estimate the degree of linear dependency between two random variables. On the other hand, mutual information methods can capture any kind of statistical dependency, but being nonparametric, they require more samples for accurate estimation.

---

**🔍 Note: Feature selection with sparse data**

If you use sparse data (i.e. data represented as sparse matrices), `chi2`, `mutual_info_regression`, `mutual_info_classif` will deal with the data without making it dense.

---

---

**⚠️ Warning**

Beware not to use a regression scoring function with a classification problem, you will get useless results. 

---

**📚 Examples:**

* [Univariate Feature Selection](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_feature_selection.html#sphx-glr-auto-examples-feature-selection-plot-feature-selection-py)
* [Comparison of F-test and mutual information](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_f_test_vs_mi.html#sphx-glr-auto-examples-feature-selection-plot-f-test-vs-mi-py)

## [Recursive feature elimination](https://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)

Given an external *estimator that assigns weights to features* (e.g., the coefficients of a linear model), the goal of recursive feature elimination ([`RFE`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE)) is to select features by **recursively considering smaller and smaller sets of features**.
* First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through any specific attribute (such as `coef_`, `feature_importances_`) or *callable*.
* Then, the least important features are pruned from current set of features. 
* 🔁 That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

[`RFECV`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV) performs RFE in a cross-validation loop to find the optimal number of features.

**📚 Examples:**
* [Recursive feature elimination](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_digits.html#sphx-glr-auto-examples-feature-selection-plot-rfe-digits-py): A recursive feature elimination example showing the relevance of pixels in a digit classification task.
* [Recursive feature elimination with cross-validation](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py): A recursive feature elimination example with automatic tuning of the number of features selected with cross-validation.

## [Feature selection using `SelectFromModel`](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-using-selectfrommodel)

`SelectFromModel` is a **meta-transformer** that can be used alongside any estimator that assigns importance to each feature through:
* a specific attribute (such as `coef_`, `feature_importances_`) or 
* via an `importance_getter()` *callable* 

after fitting.

The features are considered unimportant and removed if the corresponding importance of the feature values are below the provided `threshold` parameter.

Apart from specifying the threshold numerically, there are *built-in heuristics for finding a threshold* using a string argument.

Available heuristics are: 
* `"mean"`, 
* `"median"` and 
* *float multiples of these*, like `"0.1*mean"`.

In combination with the threshold criteria, one can use the `max_features` parameter to set a limit on the number of features to select.

For examples on how it is to be used refer to the sections below.

**📚 Example:**
* [Model-based and sequential feature selection](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_diabetes.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-diabetes-py)

### L1-based feature selection

Linear models penalized with the `L1` norm **have sparse solutions**: many of their estimated coefficients are zero. 

When the goal is to reduce the dimensionality of the data to use with another classifier, *they can be used along with `SelectFromModel`* to select the non-zero coefficients. 

In particular, sparse estimators useful for this purpose are: 
* the `Lasso` for regression, and 
* `LogisticRegression` and `LinearSVC` for classification:

In [9]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
X, y = load_iris(return_X_y=True)
print(X.shape)

(150, 4)


In [10]:
lsvc = LinearSVC(
    C=0.01, 
    penalty="l1",  # <-- l1, THIS IS KEY! 
    dual=False
).fit(X, y)

/mnt/space/miniconda3/envs/py38_playaround-sk/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [11]:
# Now set up the meta-transformer:
model = SelectFromModel(
    lsvc, 
    prefit=True
)
X_new = model.transform(X)
print(X_new.shape)

(150, 3)


ℹ️ With SVMs and logistic-regression, the parameter `C` controls the sparsity: the smaller `C` the fewer features selected. With Lasso, the higher the `alpha` parameter, the fewer features selected.

**📚 Examples:**
* [Classification of text documents using sparse features](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py): Comparison of different algorithms for document classification including L1-based feature selection.

🎓 For a discussion of "L1-recovery and compressive sensing" see [end of this section](https://scikit-learn.org/stable/modules/feature_selection.html#l1-based-feature-selection).

###  Tree-based feature selection

Tree-based estimators (see the `sklearn.tree` module and forest of trees in the `sklearn.ensemble` module) can be used to compute impurity-based feature importances, which in turn can be used to discard irrelevant features (when coupled with the `SelectFromModel` meta-transformer):

In [12]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

In [13]:
X, y = load_iris(return_X_y=True)
X.shape

(150, 4)

In [14]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
clf.feature_importances_

array([0.11040523, 0.05410085, 0.43245421, 0.40303971])

In [15]:
model = SelectFromModel(clf, prefit=True)  # Meta-transformer.
X_new = model.transform(X)
X_new.shape

(150, 2)

**📚 Examples:**
* [Feature importances with a forest of trees](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-py): example on synthetic data showing the recovery of the actually meaningful features.
* [Pixel importances with a parallel forest of trees](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances_faces.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-faces-py): example on face recognition data.

## [Sequential Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#l1-based-feature-selection)

**Sequential Feature Selection** (SFS) is available in the `SequentialFeatureSelector` transformer. SFS can be either **forward** or **backward**.

**Forward-SFS** is a greedy procedure that iteratively finds the best new feature to add to the set of selected features. Concretely: 
* We initially start with zero feature and find the one feature that *maximizes a cross-validated score* when an estimator is trained on this single feature. 
* Once that first feature is selected, we repeat the procedure by adding a new feature to the set of selected features. 
* The procedure stops when the desired number of selected features is reached, as determined by the `n_features_to_select` parameter.

**Backward-SFS** follows the same idea but works in the opposite direction: instead of starting with no feature and greedily adding features, we start with *all* the features and greedily *remove* features from the set. 

ℹ️ The `direction` parameter controls whether forward or backward SFS is used.

In general, forward and backward selection *do not yield equivalent results*. Also, one may be much faster than the other depending on the requested number of selected features: if we have 10 features and ask for 7 selected features, forward selection would need to perform 7 iterations while backward selection would only need to perform 3.

ℹ️ SFS differs from `RFE` and `SelectFromModel` in that **it does not require the underlying model to expose a `coef_` or `feature_importances_` attribute**. It may however be **slower** considering that more models need to be evaluated, compared to the other approaches. For example in backward selection, the iteration going from `m` features to `m - 1` features using k-fold cross-validation requires fitting `m * k` models, while `RFE` would require only a single fit, and `SelectFromModel` always just does a single fit and requires no iterations.

**📚 Example:**
* [Model-based and sequential feature selection](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_diabetes.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-diabetes-py)

## [Feature selection as part of a pipeline](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-as-part-of-a-pipeline)

Feature selection is usually used as a pre-processing step before doing the actual learning. 

The recommended way to do this in scikit-learn is to use a `Pipeline`:

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False, max_iter=100000))),
  ('classification', RandomForestClassifier())
])

clf.fit(X, y)

Pipeline(steps=[('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     max_iter=100000,
                                                     penalty='l1'))),
                ('classification', RandomForestClassifier())])

In this snippet we make use of a `LinearSVC` coupled with `SelectFromModel` to evaluate feature importances and select the most relevant features.

Then, a `RandomForestClassifier` is trained on the transformed output, i.e. using only relevant features. You can perform similar operations with the other feature selection methods and also classifiers that provide a way to evaluate feature importances of course. See the `Pipeline` examples for more details.